### Imports

In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx

import folium
import branca.colormap as cmap # a folium spin-off for colormapping.

import Agent as ag
import eindhovenDists as ed

Loading networks


In [ ]:
# Generate a random seed for reproducibility
seed = 42
import random
random.seed(seed)

### Simulation
The rules of the simulation are as follows:
- Each timestep t representing a day. We simulate for each Agent the course of one day
- Each agent represents a person living at a specific residence in Eindhoven
- Each agent has a list of amenity types that they visit regularly and a probability that they will visit it on any given day.
- Each agent can either walk or cycle to work. They randomly choose whether they walk or cycle each day.
- Some agents have a workplace. This can be one of the amenity types. They will always go to the same workplace with a high probability on each day.
- For the amenities they don’t work at, an agent keeps a list of the closest n amenities to their home or workplace, and go to one of those on the list based on a geometrically decreasing probability in order of proximity. We name this probability "curiosity".

In [ ]:
# Initialization

place = 'Eindhoven, Netherlands'
# Get the graph of the area
G = ox.graph_from_place(place, network_type='walk', simplify=True)
# Get the nodes and edges of the graph
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)


# Adapted from eindhovenMap.py
tags = {
    'office': ['yes'],
    'amenity': [
        'pharmacy', 'hospital', 'clinic', 'doctors', 
        'school', 'kindergarten', 'college', 'university',  
        'cafe', 'restaurant', 'bar', 'cinema', 'theatre',  
        'community_centre', 'library', 'bicycle_rental',
        'place_of_worship', 
    ],
    'leisure': [
        'park', 'fitness_centre', 'sports_centre', 'stadium', 
        'dog_park', 'pitch', 'swimming_pool'
    ]
}

# Get the amenities of the area
amenities = ox.features_from_place(place, tags=tags)

# Get the residences of the area
residences = ox.features_from_place(place, tags={'building': ['residential']})


In [ ]:
# For now just simulate a single agent
# Create a random agent

# Get the coordinates of a random residence
residence_location = residences.sample(1).geometry.values[0]
residence_coords = (residence_location.x, residence_location.y)
# Get the coordinates for a random workplace
workplace_location = amenities.sample(1).geometry.values[0]
workplace_coords = (workplace_location.x, workplace_location.y)

agent = ag.Agent(
    residence_location= residence_coords, 
    workplace_location= workplace_coords,
    seed=seed
)

agent.simulate_day()



